In [1]:
from Rooms.ArrivalRoom import ArrivalRoom
from Rooms.WaitingRoom import WaitingRoom
from EnvironmentElements.Patient import Patient
from Rooms.TreatmentRoom import TreatmentRoom
from Rooms.ExitRoom import ExitRoom
from datetime import datetime, timedelta
import time
from IPython.display import clear_output

In [3]:
# Simulation configuration
arrival_rate = 10  # Patients per minute
simulation_duration_minutes = 1

# Time between patient arrivals (in seconds)
time_between_patients_arrival = 1 / arrival_rate * 60  # Time in seconds
timenow = datetime.now()
endtime = timenow + timedelta(minutes=simulation_duration_minutes)

# Initialize simulation components
patients = []
arrival_room = ArrivalRoom()
waiting_room = WaitingRoom(6)  # Capacity 6
treatment_room = TreatmentRoom(2)  # Capacity 2
exit_room = ExitRoom()

# Initialize the start time of the simulation
simulation_start_time = datetime.now()
last_patient_arrival_time = simulation_start_time

# Main simulation loop
while timenow <= endtime:
    clear_output(wait=True)  # Clear the output for fresh updates
    timenow = datetime.now()  # Update current time
    
    # Calculate elapsed time in seconds
    elapsed_time = (timenow - last_patient_arrival_time).total_seconds()
    total_seconds_passed = (timenow - simulation_start_time).total_seconds()
    
    # Handle new patient arrivals
    if elapsed_time >= time_between_patients_arrival:
        new_patient = Patient()
        patients.append(new_patient)
        print(f"[{timenow.strftime('%H:%M:%S')}] New patient arrived (Total time: {total_seconds_passed:.2f}s).")
        
        # Update the last patient arrival time
        last_patient_arrival_time = timenow

    # Process patients in Arrival Room
    if patients:  # If there are patients waiting to be registered
        arrival_room.registerPatient(patients.pop(0))
        print(f"[{timenow.strftime('%H:%M:%S')}] Patient registered (Total time: {total_seconds_passed:.2f}s).")
        arrival_room.sendToWaitingRoom(waiting_room)

    if elapsed_time % 3 == 0 and waiting_room.getQueue():
        print(f"[{timenow.strftime('%H:%M:%S')}] Escalation in Progress! (Total time: {total_seconds_passed:.2f}s)")
        # Process patients in Waiting Room
        for waiting_patient in waiting_room.getQueue():
            waiting_room.escalate(waiting_patient)
    
    waiting_signal = waiting_room.sendToTreatmentRoom(treatment_room)
    if waiting_signal:
        print(f"[{timenow.strftime('%H:%M:%S')}] {waiting_signal} (Total time: {total_seconds_passed:.2f}s)")

    # Process patients in Treatment Room
    treatment_room.sendToExitRoom(exit_room)

    # Collect reports from Exit Room
    treated_reports = exit_room.getTreatedPatientsReports()
    if treated_reports:
        for report in treated_reports[-5:]:  # Show the last 5 reports
            print(f"[{timenow.strftime('%H:%M:%S')}] Report: {report} (Total time: {total_seconds_passed:.2f}s)")

    # Print system status
    print("\n--- Current System Status ---")
    print(f"Arrival Room Queue: {len(arrival_room.getQueue())} patients")
    print(f"Waiting Room Queue: {len(waiting_room.getQueue())} patients")
    print(f"Treatment Room: {len(treatment_room.patients)} patients")
    print(f"Exit Room Treated Patients: {len(exit_room.getTreatedPatients())}")
    print("--------------------------------")

    # Pause for a short while to simulate real-time progression
    time.sleep(.5)

# End simulation
print("\nSimulation completed. Generating final reports...")
print(exit_room.getTreatedPatientsReports())


[22:32:50] No More Patients in Waiting Room! (Total time: 60.18s)
[22:32:50] Report: {'patient_id': 6380482, 'priority_level': 'High', 'triage_category': 'Level 2: Emergent (Very Urgent)', 'waiting_time': 9e-06, 'treatment_time': 5.05802, 'total_time': 5.05807} (Total time: 60.18s)
[22:32:50] Report: {'patient_id': 1667818, 'priority_level': 'Very High', 'triage_category': 'Level 1: Resuscitation (Critical)', 'waiting_time': 4e-06, 'treatment_time': 12.135614, 'total_time': 12.13564} (Total time: 60.18s)
[22:32:50] Report: {'patient_id': 8356905, 'priority_level': 'Low', 'triage_category': 'Level 4: Less Urgent', 'waiting_time': 1.5e-05, 'treatment_time': 1.008886, 'total_time': 1.008947} (Total time: 60.18s)
[22:32:50] Report: {'patient_id': 3888657, 'priority_level': 'Medium', 'triage_category': 'Level 3: Urgent', 'waiting_time': 1.2e-05, 'treatment_time': 4.055388, 'total_time': 4.055476} (Total time: 60.18s)
[22:32:50] Report: {'patient_id': 2283718, 'priority_level': 'Very Low', '